## Build a Delft3D FM 2D model from scratch

This notebook demonstrates how to prepare a **2D Delft3D FM** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### HydroMT CLI build interface

Lets first check if the Delft3D FM model is recognized by HydroMT:

In [1]:
# this should return "dflowfm" to build DFlowFM components of Delft3D FM
# as well as the generic HydroMT models "grid_model, lumped_model, mesh_model, network_model"
!hydromt --models

model plugins:
 - dflowfm (hydromt_delft3dfm 0.1.2)
generic models (hydromt 0.9.0):
 - grid_model
 - vector_model
 - mesh_model
 - network_model



Using the **HydroMT build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [2]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT

  Build models from scratch.

  Example usage: --------------

  To build a wflow model for a subbasin using a point coordinates snapped to
  cells with upstream area >= 50 km2 hydromt build wflow /path/to/model_root
  -i /path/to/wflow_config.ini  -r "{'subbasin': [-7.24, 62.09], 'uparea':
  50}" -d deltares_data -d /path/to/data_catalog.yml -v

  To build a sfincs model based on a bbox hydromt build sfincs
  /path/to/model_root  -i /path/to/sfincs_config.ini  -r "{'bbox':
  [4.6891,52.9750,4.9576,53.1994]}"  -d /path/to/data_catalog.yml -v

Options:
  --opt TEXT               Method specific keyword arguments, see the method
                           documentation of the specific model for more
                           information about the arguments.
  -i, --config PATH        Path to hydroMT configuration file, for the model
                           specific implementation.
  -r, --region TEXT        Set the region for which to 

### Data for model setup

Most of the time, the `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default, the pre-defined `artifact_data` catalog containing some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data_existing_cat.html).
This example data is a based on the data which is available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `-d deltares_data` pre-defined catalog. 

More information on how to write a data catalog yml file can be found in the [HydroMT core docs](https://deltares.github.io/hydromt/latest/user_guide/data_prepare_cat.html) 

> **NOTE**: In HydroMT-Delft3D FM, an additionnal data catalog with all default parameters values is always used [parameters_data.yml](https://github.com/Deltares/hydromt_delft3dfm/tree/main/hydromt_delft3dfm/data/parameters_data.yml). It contains default values for 1D network elements and structures as well as default 2D landuse paramter mapping tables.

### Model setup configuration

The HydroMT configuration file contains the model setup configuration and determines which methods are used to prepare the different components of a Delft3D FM model and in which order and optionally sets non-default arguments for each method. This configuration is passed to HydroMT using `-i <path_to_configuration_file>`. The supported format are either YAML or INI. We have prepared several example yml-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_delft3dfm/tree/main/examples) and from the [docs (building a model)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_build). 

Each section, before indent, (e.g. setup_mesh2d) corresponds to a model method. All model methods are explained in the [docs (model components)](https://deltares.github.io/hydromt_delft3dfm/latest/user_guide/dflowfm_model_setup.html). The `global` section contains direct model initialisation properties.

We will load the default dflowfm build 2D yaml file for inspection:

In [3]:
fn_yml = "dflowfm_build2d.yml"
with open(fn_yml, "r") as f:
    txt = f.read()
print(txt)

global:
  crs: 3857

setup_mesh2d:
  region: 
    bbox: [12.4331, 46.4661, 12.5212, 46.5369]
  res: 500

setup_maps_from_rasterdataset:
  raster_fn: merit_hydro
  variables: ["elevtn"]
  fill_method: nearest
  interpolation_method: nearestNb

setup_maps_from_raster_reclass:
  raster_fn: vito
  reclass_table_fn: vito_mapping
  reclass_variables: ['roughness_manning', 'infiltcap']
  interpolation_method: triangulation

setup_rainfall_from_constant:
  constant_value: 150



Looking at this file, you see that we will prepare the following information for our model:

- [global](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.html#hydromt_delft3dfm.DFlowFMModel): our model will be defined in the projected *crs* WGS84 EPSG 3857.
- [setup_mesh2d](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_mesh2d.html): we will prepare a regular grid for the region bounding box [12.4331, 46.4661, 12.5212, 46.5369] of *500***500* meters resolution.
- [setup_maps_from_rasterdataset](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_maps_from_rasterdataset.html): prepares gridded elevation (bedlevel) from *merit_hydro*. Here elevation is extracted from *merit_hydro* and any no data is filled using *nearest* method. When running, the Delft3D FM kernel will interpolate it to the mesh 2D grid using the *nearestNb* method.
- [setup_maps_from_raster_reclass](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_maps_from_raster_reclass.html): prepares gridded Manning roughness and infiltration capacity data from landuse, by reclassifying the landuse categories in *vito* landuse data, using parameters values defined per landuse category in *vito_mapping* table. When running, the Delft3D FM kernel will interpolate it to the mesh 2D grid using the *triangulation* method.
- [setup_rainfall_from_constant](https://deltares.github.io/hydromt_delft3dfm/latest/_generated/hydromt_delft3dfm.DFlowFMModel.setup_rainfall_from_constant.html): prepares a constant rainfall rate of *150* mm/day as meteo forcing to the model. 
Feel free to use the links of each components to know more about how each function works and what are all the available settings.

> **NOTE**: 
 In this example, we apply region argument in setup_mesh2d. The region bbox are in WGS 84 as required, but the user should specify the dflowfm model destination CRS in the `global` section in order to allow to use grid definition data (rivers, pipes) from different data sources or CRS.  

> **NOTE**:
  If you need to use the same function twice, for example you have raster data for elevation and for roughness, you can list the `setup_maps_from_rasterdataset` as many times as you wish, by adding a repeating the method and adding a number at the end of the function name. Example: setup_maps_from_rasterdataset, setup_maps_from_rasterdataset1, setup_maps_from_rasterdataset2 etc.

### Building a 2D Delft3D FM model 

In [4]:
# NOTE: copy this line (without !) to your shell for more direct feedback - should take a couple of seconds to run
!hydromt build dflowfm "./build/dflowfm_2d" -i dflowfm_build2d.yml -d artifact_data --fo -vv

2023-10-25 13:46:41,322 - build - log - DEBUG - Writing log messages to new file C:\SnapVolumesTemp\MountPoints\{45c63495-0000-0000-0000-100000000000}\{79DE0690-9470-4166-B9EE-4548DC416BBD}\SVROOT\DATA\hydromt_delft3dfm\examples\build\dflowfm_2d\hydromt.log.
2023-10-25 13:46:41,323 - build - log - INFO - HydroMT version: 0.9.0
2023-10-25 13:46:41,323 - build - main - INFO - Building instance of dflowfm model at C:\SnapVolumesTemp\MountPoints\{45c63495-0000-0000-0000-100000000000}\{79DE0690-9470-4166-B9EE-4548DC416BBD}\SVROOT\DATA\hydromt_delft3dfm\examples\build\dflowfm_2d.
2023-10-25 13:46:41,323 - build - main - INFO - User settings:
2023-10-25 13:46:42,961 - build - data_catalog - INFO - Reading data catalog archive artifact_data v0.0.8
2023-10-25 13:46:42,966 - build - data_catalog - INFO - Parsing data catalog from C:\Users\veenstra\.hydromt_data\artifact_data\v0.0.8\data_catalog.yml
2023-10-25 13:46:47,660 - build - model_api - INFO - Initializing dflowfm model from hydromt_delft

The nodata value None is not in the reclass table.None will be used for the params.
INFO:build:Initialising empty dimr file
INFO:build:Adding dflowfm component to dimr config
INFO:build:Writing model dimr file to C:\SnapVolumesTemp\MountPoints\{45c63495-0000-0000-0000-100000000000}\{79DE0690-9470-4166-B9EE-4548DC416BBD}\SVROOT\DATA\hydromt_delft3dfm\examples\build\dflowfm_2d\dimr_config.xml


The example above means the following: run **hydromt build** with:

* `dflowfm` : i.e. build a FM model
* `./build/dflowfm_2d` : output model folder
* `-i dflowfm_build1d.yml`: model build configuration file
* `-d artifact_data`: data catalog to use. Here the default artifact data catalog with global data for Northern Italy.
* `--fo`: force overwritting in case a model already exists in the output folder.
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug and info messages are provided rather than only warnings and errors.

> NOTE: You can use as many data catalogs as you need within the same command line, by repeating the `-d` options and name of the catalog to use several times.

Next we check which files have been created.

In [5]:
import os

root = "./build/dflowfm_2d/"
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        print(f" - {name}")

./build/dflowfm_2d/
 - dimr_config.xml
 - hydromt.log
 - hydromt_data.yml
./build/dflowfm_2d/dflowfm
 - bnd.ext
 - DFlowFM.mdu
 - fieldFile.ini
 - fm_net.nc
 - meteo_boundaryconditions.bc
./build/dflowfm_2d/geoms
 - mesh2d.geojson
 - region.geojson
./build/dflowfm_2d/maps
 - bedlevel.tif
 - frictioncoefficient.tif
 - InfiltrationCapacity.tif


The model root should contain three files and four folders:

- dimr_config.xml: Configuration file to execute Delft3D FM in Deltares Integrated Model Runner (DIMR).
- hydromt.log: log saved by HydroMT on model building steps.
- hydromt_data.yml: information saved by HydroMT on used datasets for reproducibility.
- **dflowfm** folder: dflowfm files, here for our 2D model. Contains all necessary information to run a 2D model including the mesh2d, and settings and paths to the mesh2d data files in the maps folder. The configuration MDU file is also created.
- **geoms** folder: folder containing a copy of the dflowfm data in an easily readable GIS format rather than the ini or UGRID netcdf file.
- **maps** folder: folder containing regular gridded data that Delft3D FM can interpolate to the 2D mesh grid. Here we see our elevation or bedlevel data, as well as the roughness / friction and infiltration capacity maps.

### Visualize and/or inspect model schematization

* We can now load the **dimr_config.xml** into **Delft3D FM GUI**.
* The **dflowfm plot** [example notebook](https://deltares.github.io/hydromt_delft3dfm/latest/_examples/plot_dflowfm_mesh.html) contains scripts to visualize your model in python: 